In [1]:
import os
import sys
import pickle
import json
from collections import defaultdict
from tqdm.notebook import tqdm
import numpy as np
from itertools import product
import random

sys.path.append('../../..')
from pygraphs.measure import kernels

In [2]:
shuffle = lambda x: sorted(x, key=lambda k: random.random())

# kkmeans init experiments, results

Final goal: compare both different initialization (one, all, k-means++, any) and initialization quality measures (inertia, modularity) 

In [3]:
CACHE_ROOT = '/home/illusionww/Documents/HDD/phd/pygraphs/kkmeans_init_datasets_modularity/by_column_and_kernel'
columns = [
    'cora_DB',
    'cora_EC',
    'cora_HA',
    'cora_HCI',
    'cora_IR',
    'cora_Net',
    'dolphins',
    'eu-core',
    'eurosis',
    'football',
    'karate',
    'news_2cl_1',
    'news_2cl_2',
    'news_2cl_3',
    'news_3cl_1',
    'news_3cl_2',
    'news_3cl_3',
    'news_5cl_1',
    'news_5cl_2',
    'news_5cl_3',
    'polblogs',
    'polbooks',
    'sp_school_day_1',
    'sp_school_day_2',
]
kernels_names = [
    'Katz', 'logKatz',
    'For', 'logFor',
    'Comm', 'logComm',
    'Heat', 'logHeat',
    'NHeat', 'logNHeat',
    'SCT', 'SCCT',
    'RSP', 'FE',
    'PPR', 'logPPR',
    'ModifPPR', 'logModifPPR',
    'HeatPR', 'logHeatPR',
    'DF', 'logDF',
    'Abs', 'logAbs',
    'SP-CT'
]

In [4]:
# test
column_str = columns[0]
kernel_name = 'Comm'
with open(f'{CACHE_ROOT}/{column_str}_{kernel_name}_results.pkl', 'rb') as f:
    data = pickle.load(f)
data[0.0][0]

{'labels': array([3, 3, 3, ..., 3, 3, 3]),
 'inertia': array(7029.0024, dtype=float32),
 'modularity': array([-6.830985e-06], dtype=float32),
 'init': 'one',
 'score_ari': 0.00046596517909929035,
 'score_nmi': 0.039346568477725836}

## Инициализация, дающая наилучший результат

Для каждого графа определяем наилучшую инициализацию **для каждого параметра** (по inertia, ARI и NMI). После этого считаем статистику внутри каждого графа, сколько побед у каждого типа инициализаций (one, all, k-kmeans++), выбираем победителя для графа. После этого считаем статистику для всех графов, выбираем победителя. Делаем это для каждой меры и каждого сетапа генерации графов

In [5]:
def group_inits(param_results):
    """
    * Сначала группируем инициализации по one, all, k-kmeans++, any,
    * Потом для каждой группы выбираем лучшую согласно inertia или modularity
    """
    one_inits = shuffle([x for x in param_results if x['init'] == 'one'])
    all_inits = shuffle([x for x in param_results if x['init'] == 'all'])
    kmp_inits = shuffle([x for x in param_results if x['init'] == 'k-means++'])
    any_inits = one_inits[:10] + all_inits[:10] + kmp_inits[:10]
    any2_inits = all_inits[:15] + kmp_inits[:15]
    if len(one_inits) == 0 or len(all_inits) == 0 or len(kmp_inits) == 0 or len(any_inits) == 0 \
        or len(any2_inits) == 0:
        return None
        
    bestby = {
        'one': {'inertia': one_inits[0], 'modularity': one_inits[0]},
        'all': {'inertia': all_inits[0], 'modularity': all_inits[0]},
        'kmp': {'inertia': kmp_inits[0], 'modularity': kmp_inits[0]},
        'any': {'inertia': any_inits[0], 'modularity': any_inits[0]},
        'any2': {'inertia': any2_inits[0], 'modularity': any2_inits[0]},
    }
    for init_name, inits in {'one': one_inits, 'all': all_inits, 'kmp': kmp_inits, 'any': any_inits,
                             'any2': any2_inits}.items():
        for init in inits:
            if init['inertia'] < bestby[init_name]['inertia']['inertia']:  # choose best by inertia
                bestby[init_name]['inertia'] = init
            if init['modularity'] > bestby[init_name]['modularity']['modularity']:  # choose best by modularity
                bestby[init_name]['modularity'] = init
    return bestby

In [8]:
def perform_column_kernel(data, score_name='score_ari'):
    reaching_single = {
        'inertia_one': 0, 'inertia_all': 0, 'inertia_kmp': 0,
        'modularity_one': 0, 'modularity_all': 0, 'modularity_kmp': 0,
    }
    reaching_all = {
        'inertia_one': 0, 'inertia_all': 0, 'inertia_kmp': 0, 'inertia_any': 0, 'inertia_any2': 0,
        'modularity_one': 0, 'modularity_all': 0, 'modularity_kmp': 0, 'modularity_any': 0, 'modularity_any2': 0
    }
    best_score = {
        'inertia_one': 0, 'inertia_all': 0, 'inertia_kmp': 0, 'inertia_any': 0, 'inertia_any2': 0,
        'modularity_one': 0, 'modularity_all': 0, 'modularity_kmp': 0, 'modularity_any': 0, 'modularity_any2': 0
    }
    # choose best score accross kernel param
    for param, param_results in data.items():
        bestby = group_inits(param_results)
        if bestby is not None:
            for unsup, init_name in product(['inertia', 'modularity'], ['one', 'all', 'kmp', 'any', 'any2']):
                try:
                    score = bestby[init_name][unsup][score_name]
                except Exception as e:
                    print(f'graph_idx: {graph_idx}, param: {param}, init: {init_name}, unsup: {unsup}')
                    print(f'best_init: {bestby[init_name][unsup]}')
                    raise e
                if score > best_score[f'{unsup}_{init_name}']:
                    best_score[f'{unsup}_{init_name}'] = score

    # add 1 if setup reaches maximum
    max_single = max([best_score[k] for k in ['inertia_one', 'inertia_all', 'inertia_kmp',
                                              'modularity_one', 'modularity_all', 'modularity_kmp']])
    max_all = max(best_score.values())
    for unsup, init_name in product(['inertia', 'modularity'], ['one', 'all', 'kmp']):
        reaching_single[f'{unsup}_{init_name}'] += best_score[f'{unsup}_{init_name}'] == max_single
    for unsup, init_name in product(['inertia', 'modularity'], ['one', 'all', 'kmp', 'any', 'any2']):
        reaching_all[f'{unsup}_{init_name}'] += best_score[f'{unsup}_{init_name}'] == max_all

    return best_score, reaching_single, reaching_all

In [9]:
results = defaultdict(lambda: dict())
for column, kernel_name in tqdm(list(product(columns, kernels_names))):
    column_str = column
    filename = f'{column_str}_{kernel_name}_results.pkl'
#     print(f'{column_str}, {kernel_name}')
    with open(f'{CACHE_ROOT}/{filename}', 'rb') as f:
        data = pickle.load(f)
    best_score, reaching_single, reaching_all = perform_column_kernel(data)
    results[column_str][kernel_name] = {
        'mean_scores': best_score,
        'reaching_single': reaching_single,
        'reaching_all': reaching_all,
    }

In [10]:
with open('results.pkl', 'wb') as f:
    pickle.dump(dict(results), f)

In [11]:
with open('mean_scores.tsv', 'w') as f:
    f.write('\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
            'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
            f.write(f'{column}\t')
    f.write('\n\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
            'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
            f.write(f'{init_name}\t')
    f.write('\n')
    for kernel_name in kernels_names:
        f.write(f'{kernel_name}\t')
        for column in columns:
            column_str = column
            for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
                'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
                f.write(f'{results[column_str][kernel_name]["mean_scores"][init_name]}\t')
        f.write('\n')

In [12]:
# print as table

with open('reaching_single.tsv', 'w') as f:
    f.write('\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp',
            'modularity_one', 'modularity_all', 'modularity_kmp']:
            f.write(f'{column}\t')
    f.write('\n\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp',
            'modularity_one', 'modularity_all', 'modularity_kmp']:
            f.write(f'{init_name}\t')
    f.write('\n')
    for kernel_name in kernels_names:
        f.write(f'{kernel_name}\t')
        for column in columns:
            column_str = column
            for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp',
                'modularity_one', 'modularity_all', 'modularity_kmp']:
                f.write(f'{results[column_str][kernel_name]["reaching_single"][init_name]}\t')
        f.write('\n')

In [13]:
# print as table

with open('reaching_all.tsv', 'w') as f:
    f.write('\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
            'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
            f.write(f'{column}\t')
    f.write('\n\t')
    for column in columns:
        for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
            'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
            f.write(f'{init_name}\t')
    f.write('\n')
    for kernel_name in kernels_names:
        f.write(f'{kernel_name}\t')
        for column in columns:
            column_str = column
            for init_name in ['inertia_one', 'inertia_all', 'inertia_kmp', 'inertia_any', 'inertia_any2',
                'modularity_one', 'modularity_all', 'modularity_kmp', 'modularity_any', 'modularity_any2']:
                f.write(f'{results[column_str][kernel_name]["reaching_all"][init_name]}\t')
        f.write('\n')